In [ ]:
import numpy as np
from numpy import load

import pandas as pd

from google.colab import drive

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load array
X_train = load('/content/drive/My Drive/images_for_training/X_train.npy')
y_train = load('/content/drive/My Drive/images_for_training/y_train.npy')
X_test = load('/content/drive/My Drive/images_for_training/X_test.npy')
y_test = load('/content/drive/My Drive/images_for_training/y_test.npy')


In [ ]:
# model = Sequential()
# model.add(
#         Conv2D(
#             26,
#             kernel_size = (5, 5),
#             padding = 'valid',
#             activation = 'relu',
#             input_shape = (100, 100, 3)
#         )
# )
# model.add(
#         Conv2D(
#             26, 
#             kernel_size = (5,5),
#             padding = "valid",
#             activation='relu'
#         )
#     )
# model.add(
#         MaxPooling2D(
#             pool_size = (2, 2)
#         )
# )
# model.add(
#         Dropout(
#             0.7114247520786349
#         )
# )
# model.add(Flatten())
# model.add(
#         Dense(
#                 512
#         )
# )
# model.add(
#         Activation(
#             'tanh'
#         )
# )
# model.add(
#         Dropout(
#             0.5623815057565213
#         )
# )
# model.add(Dense(24, activation='softmax'))


In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu', input_shape=(200, 200, 1)))
model.add(Conv2D(32, kernel_size=4, strides=2, activation='relu'))
model.add(Dropout(.5))
model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
model.add(Dropout(.5))
model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(256, activation='tanh'))
model.add(Dense(26, activation='softmax'))

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 98, 98, 64)        65600     
_________________________________________________________________
dropout (Dropout)            (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 95, 95, 128)       131200    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 128)       262272    
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 46, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 43, 43, 256)       5

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
callback = EarlyStopping(monitor='loss', patience=3)

result = model.fit(X_train, y_train, epochs=10, verbose=2, validation_split=0.1, callbacks=[callback])


Epoch 1/10
1755/1755 - 121s - loss: 3.2593 - accuracy: 0.0365 - val_loss: 3.2584 - val_accuracy: 0.0380
Epoch 2/10
1755/1755 - 121s - loss: 3.2584 - accuracy: 0.0387 - val_loss: 3.2583 - val_accuracy: 0.0333
Epoch 3/10
1755/1755 - 121s - loss: 3.2584 - accuracy: 0.0385 - val_loss: 3.2585 - val_accuracy: 0.0333
Epoch 4/10
1755/1755 - 121s - loss: 3.2584 - accuracy: 0.0383 - val_loss: 3.2587 - val_accuracy: 0.0349
Epoch 5/10
1755/1755 - 121s - loss: 3.2584 - accuracy: 0.0386 - val_loss: 3.2585 - val_accuracy: 0.0349
Epoch 6/10
1755/1755 - 121s - loss: 3.2584 - accuracy: 0.0384 - val_loss: 3.2588 - val_accuracy: 0.0349
Epoch 7/10
1755/1755 - 121s - loss: 3.2584 - accuracy: 0.0371 - val_loss: 3.2586 - val_accuracy: 0.0349


In [ ]:
model.save('/content/drive/My Drive/model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/model/assets


In [ ]:
model.save('/content/drive/My Drive/model/model_1.h5', save_format='h5')

In [ ]:
validation_acc = np.amax(result.history['val_accuracy'])
print('Best validation acc of epoch:', validation_acc)



Best validation acc of epoch: 0.9906542301177979


In [ ]:
# alphabet = ['A']

alphabet = ['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y']

test_samples = []
test_letters = []
for letter in alphabet:
  img = f'/content/drive/My Drive/images_for_testing/{letter}/Test-{letter}1.png'
  img = image.load_img(img, target_size = (100, 100, 3), color_mode = "rgb")
  img = image.img_to_array(img)
  img = img/255
  test_samples.append(img)
  test_letters = alphabet

test_samples.append(img)
X = np.array(test_samples)

label_encoder = LabelEncoder()
label_encoder.fit(test_letters)

print('#################')
print(X)
print('#################')

encoded_predictions = model.predict_classes(X[:])
print(encoded_predictions)

prediction_labels = label_encoder.inverse_transform(encoded_predictions)
print('######## PREDICTION #########')
print(prediction_labels)
print('######## PREDICTION #########')


#################
[[[[0.9843137  0.98039216 0.98039216]
   [0.9843137  0.98039216 0.98039216]
   [0.9843137  0.98039216 0.98039216]
   ...
   [0.2509804  0.13333334 0.14901961]
   [0.25490198 0.13333334 0.09803922]
   [0.24705882 0.13333334 0.08235294]]

  [[0.9843137  0.98039216 0.98039216]
   [0.9843137  0.98039216 0.98039216]
   [0.9843137  0.98039216 0.98039216]
   ...
   [0.2509804  0.11372549 0.14901961]
   [0.2509804  0.10588235 0.09803922]
   [0.23921569 0.10196079 0.04705882]]

  [[0.9843137  0.98039216 0.98039216]
   [0.9843137  0.98039216 0.98039216]
   [0.9843137  0.98039216 0.98039216]
   ...
   [0.2509804  0.10980392 0.18431373]
   [0.2509804  0.12156863 0.20784314]
   [0.23921569 0.11764706 0.23137255]]

  ...

  [[0.11764706 0.08235294 0.07058824]
   [0.06666667 0.07450981 0.05490196]
   [0.06666667 0.07058824 0.04705882]
   ...
   [0.41568628 0.34117648 0.03529412]
   [0.41568628 0.68235296 0.28627452]
   [0.5921569  0.6156863  0.5372549 ]]

  [[0.09803922 0.06666667 0